# Feature #1: Import custom NuGet or PyPI packages
### C#: Use the syntax: ``#r "nuget: PackageName, OptionalVersionNumber"``. 


In [10]:
#r "nuget: Bogus"
#r "nuget: System.Linq"
#r "nuget: System.Linq.Expressions"

### Python: Use pip install

In [ ]:
// import sys
// !{sys.executable} -m pip install bokeh==1.4.0 --user

# Feature #2: Use built-in Cosmos SDK

In [4]:
using System;
using Microsoft.Azure.Cosmos;
using System.Collections;

// Initialize a new instance of CosmosClient using the built-in account endpoint and key parameters
CosmosClient cosmosClient = new CosmosClient(Cosmos.Endpoint, Cosmos.Key);

// Create a new database and a new container
Microsoft.Azure.Cosmos.Database database = await cosmosClient.CreateDatabaseIfNotExistsAsync("CosmosIOTDemo");
Container container = await database.CreateContainerIfNotExistsAsync("VehicleData", "/vin");

Display.AsMarkdown(@"
Created database CosmosIOTDemo and container VehicleData. You can see these new resources by refreshing your resource pane under the Data section.
");


Created database CosmosIOTDemo and container VehicleData. You can see these new resources by refreshing your resource pane under the Data section.


### Tip: Want to try out custom CosmosClient options?

In [13]:
using Microsoft.Azure.Cosmos;

// Configure CosmosClientOptions
var cosmosClientOptions = new CosmosClientOptions
{
    ApplicationName = "ContosoNotebookAppName",
    ApplicationRegion = Regions.WestUS2, // By default, this is the region you first created your account in
    MaxRetryAttemptsOnRateLimitedRequests = 9, // By default, this value is 9
};

var client = new CosmosClient(Cosmos.Endpoint, Cosmos.Key, cosmosClientOptions);

### Tip: Use built-in variable display

In [14]:
cosmosClientOptions

Microsoft.Azure.Cosmos.CosmosClientOptions

# Feature #3: Upload JSON file into a container

In [15]:
%%upload --databaseName RetailDemo --containerName WebsiteData --url https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/websiteData.json

Documents successfully uploaded to WebsiteData 
Total number of documents imported: 
  Success: 2654 
  Failure: 0 
Total time taken : 00:00:05 hours 
Total RUs consumed : 27309.660000001593

# Feature #4: Use built-in visualizations

### Represent data type with C# class

In [ ]:
using System;

class TelemetryEvent
{
    public string id { get; set; }
    
    public string vin { get; set; }

    public string eventName { get; set; }

    public string vehicleType { get; set; }
        
    public string fuelType {get; set; }    

    public double s1 { get; set; }

    public double s2 { get; set; }

    public double s3 { get; set; }

    public DateTime timestamp { get; set; }

    public string date { get; set; }

    public string region { get; set; }

}

### Generate and write sample data

In [5]:
using System.Linq;
using System.Collections.Generic;
using System.Linq.Expressions;
using System;
using Bogus;

static List<TelemetryEvent> GenerateTelemetryEvents(int numberOfEventsToGenerate)
{
    var eventTypes = new string[] { "Harsh_break", "Airbag_deploy", "Check_engine_light" };

    var faker = new Faker();
    
    var vinList = Enumerable.Range(1, 10) //Generate 10 different vehicle ids
    .Select(_ => _.ToString())
    .ToList();
    
    var telemetryEvent = new Faker<TelemetryEvent>()
    .StrictMode(true)
    //Generate telemetry event
    .RuleFor(o => o.id, f => Guid.NewGuid().ToString())
    .RuleFor(o => o.vin, f => f.PickRandom(vinList))
    .RuleFor(o => o.eventName, f => f.PickRandom(eventTypes))
    .RuleFor(o => o.vehicleType, f => f.Vehicle.Type())
    .RuleFor(o => o.fuelType, f => f.Vehicle.Fuel())
    .RuleFor(o => o.timestamp, f => DateTime.Now) // just today's date
    .RuleFor(o => o.date, (f, m) => $"{m.timestamp.ToString("yyyy-MM-dd")}")
    .RuleFor(o => o.s1, f => f.Random.Double(1, 2000))
    .RuleFor(o => o.s2, f => f.Random.Double(1, 2000))
    .RuleFor(o => o.s3, f => f.Random.Double(1, 2000))
    .RuleFor(o => o.region, f => f.Address.Country());
    
    var events = telemetryEvent.Generate(numberOfEventsToGenerate);
       
    return events;

}

var telemetryEvents = GenerateTelemetryEvents(100);

foreach (TelemetryEvent telemetryEvent in telemetryEvents)
{
    var itemResponse = await container.CreateItemAsync<TelemetryEvent>(telemetryEvent);
}

Display.AsMarkdown(@"
Created 100 items in VehicleData container. 
");



Created 100 items in VehicleData container. 


### Query and visualize data

In [18]:
using System.Collections.Generic;

QueryDefinition queryDefinition = new QueryDefinition("SELECT TOP 100 * FROM c WHERE c.vin = @vin")
                                      .WithParameter("@vin", "8");

FeedIterator<TelemetryEvent> queryResultSetIterator = container.GetItemQueryIterator<TelemetryEvent>(queryDefinition);

List<TelemetryEvent> telemetryEvents = new List<TelemetryEvent>();

while (queryResultSetIterator.HasMoreResults)
{
    FeedResponse<TelemetryEvent> currentResultSet = await queryResultSetIterator.ReadNextAsync();
    foreach (TelemetryEvent telemetryEvent in currentResultSet)
    {
        telemetryEvents.Add(telemetryEvent);
    }
}

telemetryEvents

System.Collections.Generic.List`1[Submission#9+TelemetryEvent]

# Feature #5: Query data into Pandas dataframe (Python)